<a href="https://colab.research.google.com/github/kordusmonika/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

In [16]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score


%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd /content/drive/My Drive/Colab Notebooks/dataworkshop_matrix/dw_matrix_road_sign/dw_matrix_road_sign/data 


/content/drive/My Drive/Colab Notebooks/dataworkshop_matrix/dw_matrix_road_sign/dw_matrix_road_sign/data


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]  #(32, 32, 3)
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train) ),
      callbacks= [tensorboard_callback] #visualisation
    
  )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax( y_pred_prob, axis=1 )

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3,3), activation='relu' ),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3,3), activation='relu' ),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax'),

])


In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train )

Epoch 1/5
272/272 [==============================] - 9s 34ms/step - loss: 2.6124 - accuracy: 0.2717 - val_loss: 1.2522 - val_accuracy: 0.5859
Epoch 2/5
272/272 [==============================] - 9s 32ms/step - loss: 0.9628 - accuracy: 0.6880 - val_loss: 0.2048 - val_accuracy: 0.9373
Epoch 3/5
272/272 [==============================] - 9s 32ms/step - loss: 0.4012 - accuracy: 0.8788 - val_loss: 0.1093 - val_accuracy: 0.9666
Epoch 4/5
272/272 [==============================] - 9s 32ms/step - loss: 0.2499 - accuracy: 0.9264 - val_loss: 0.0551 - val_accuracy: 0.9836
Epoch 5/5
272/272 [==============================] - 9s 32ms/step - loss: 0.1787 - accuracy: 0.9470 - val_loss: 0.0470 - val_accuracy: 0.9853


In [12]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.1608 - accuracy: 0.9558


[0.16081079840660095, 0.9557822942733765]

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch size', 128) ),
      epochs = 7,
      verbose = 0,  
  )
  
  #we want out function to be minimal. Accurracy goes to 1
  accuracy = model.evaluate(X_test, y_test )[0]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [19]:
space = {
    'batch_size': hp.quniform('batch_size', 50,200,20)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    1,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.2064 - accuracy: 0.9062

 12/138 [=>............................]
 - ETA: 0s - loss: 0.1015 - accuracy: 0.9661

 22/138 [===>..........................]
 - ETA: 0s - loss: 0.6731 - accuracy: 0.9304

 33/138 [======>.......................]
 - ETA: 0s - loss: 0.8104 - accuracy: 0.9384

 43/138 [========>.....................]
 - ETA: 0s - loss: 0.6315 - accuracy: 0.9506

 51/138 [==========>...................]
 - ETA: 0s - loss: 0.5424 - accuracy: 0.9553


In [0]:
#Make our function more dynamic - we are looking for dropouts

def get_model(params):
  return Sequential([
    Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3,3), activation='relu' ),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(128, kernel_size=(3,3), activation='relu' ),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),
    
    Dense(num_classes, activation='softmax'),

])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=128, #int(params.get('batch size', 128) ),
      epochs = 5,
      verbose = 0,  
  )
  
  # function --> min. Accurracy goes to 1
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [31]:
space = {
    'batch_size': hp.quniform('batch_size', 50,200,20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3,0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3,0.7),

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.4731882516575788, 'dropout_cnn_block_three': 0.45437200972185515, 'dropout_cnn_block_two': 0.41449938048309104, 'dropout_dense_block_one': 0.30007791661935107, 'dropout_dense_block_two': 0.4679861029857437}
accuracy=0.9630385637283325
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.44622950778600123, 'dropout_cnn_block_three': 0.4424594692015976, 'dropout_cnn_block_two': 0.34979393494977085, 'dropout_dense_block_one': 0.32067633663780637, 'dropout_dense_block_two': 0.5415399633691639}
accuracy=0.9539682269096375
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3853293588412873, 'dropout_cnn_block_three': 0.49840758665275775, 'dropout_cnn_block_two': 0.3191401089007994, 'dropout_dense_block_one': 0.3991781933829543, 'dropout_dense_block_two': 0.6231854562884772}
accuracy=0.9501133561134338
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.38182199928324667, 'dropout_cnn_block_three': 0.48256429256208133, 'dropout_cnn_block_two': 0.4884681

#Best loss : ~98%